In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets,transforms,models
import matplotlib.pyplot as plt
import time
import os
import copy

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [7]:
mean = np.array([0.485,0.456,0.406])
std = np.array([0.229,0.224,0.225])

data_transforms = {
    'train':transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]),
    'val':transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ])
}

In [9]:
data_dir = 'train_dataset'
sets = ['train','val']
image_datasets = {x:datasets.ImageFolder(os.path.join(data_dir,x),
                                         data_transforms[x]) 
                  for x in ['train','val']}
dataloaders = {x:torch.utils.data.DataLoader(image_datasets[x],batch_size=4,
                                             shuffle=True,num_workers=0)
                for x in ['train','val']}


In [10]:
dataset_sizes = {x:len(image_datasets[x]) for x in ['train','val']}
class_names = image_datasets['train'].classes
print(class_names)

['1', '2', '3']


In [11]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-'*10)
        
        #Training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            #Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                #forward
                #track history only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _,preds = torch.max(outputs,1)
                    loss = criterion(outputs,labels)
                    
                    #backward + optimize only in train
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                
                #statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            #deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        print()
        
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')
            
    #load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
pretrained_model = models.resnet18(pretrained=True)

#exchange the last layer
num_ftrs = pretrained_model.fc.in_features
pretrained_model.fc = nn.Linear(num_ftrs, 3)
pretrained_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pretrained_model.parameters(),lr=0.001)

#scheduler
step_lr_scheduler = lr_scheduler.StepLR(optimizer,step_size=7,gamma=0.1)

model_ft = train_model(pretrained_model,criterion,optimizer,step_lr_scheduler,num_epochs=25)

/home/anas/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/anas/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/anas/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100.0%


Epoch 0/24
----------
train Loss: 1.1276 Acc: 0.3556
val Loss: 1.1586 Acc: 0.4000

Epoch 1/24
----------
train Loss: 1.0543 Acc: 0.4667
val Loss: 0.9213 Acc: 0.4667

Epoch 2/24
----------
train Loss: 1.0378 Acc: 0.4222
val Loss: 0.7630 Acc: 0.6667

Epoch 3/24
----------
train Loss: 0.8691 Acc: 0.6889
val Loss: 0.7010 Acc: 0.6667

Epoch 4/24
----------
train Loss: 0.7648 Acc: 0.7111
val Loss: 0.6196 Acc: 0.8000

Epoch 5/24
----------
train Loss: 0.7176 Acc: 0.7556
val Loss: 0.5191 Acc: 0.9333

Epoch 6/24
----------
train Loss: 0.6985 Acc: 0.7778
val Loss: 0.5054 Acc: 0.8000

Epoch 7/24
----------
train Loss: 0.6122 Acc: 0.8000
val Loss: 0.4774 Acc: 0.8667

Epoch 8/24
----------
train Loss: 0.7046 Acc: 0.7778
val Loss: 0.4666 Acc: 0.8667

Epoch 9/24
----------
train Loss: 0.6961 Acc: 0.7333
val Loss: 0.4575 Acc: 0.8667

Epoch 10/24
----------
train Loss: 0.6356 Acc: 0.8000
val Loss: 0.4574 Acc: 0.8667

Epoch 11/24
----------
train Loss: 0.5861 Acc: 0.8444
val Loss: 0.4737 Acc: 0.9333

Ep

In [ ]:
pretrained_model = models.resnet18(pretrained=True)

for param in pretrained_model.parameters():
    param.requires_grad = False
    
#exchange the last layer
num_ftrs = pretrained_model.fc.in_features
pretrained_model.fc = nn.Linear(num_ftrs, 3)
pretrained_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pretrained_model.parameters(),lr=0.001)

#scheduler
step_lr_scheduler = lr_scheduler.StepLR(optimizer,step_size=7,gamma=0.1)

model_ft = train_model(pretrained_model,criterion,optimizer,step_lr_scheduler,num_epochs=25)

Epoch 0/24
----------
train Loss: 1.1869 Acc: 0.4222
val Loss: 1.0917 Acc: 0.4667

Epoch 1/24
----------
train Loss: 1.0690 Acc: 0.4444
val Loss: 0.9793 Acc: 0.6000

Epoch 2/24
----------
train Loss: 1.0324 Acc: 0.5556
val Loss: 0.9712 Acc: 0.6000

Epoch 3/24
----------
train Loss: 0.9584 Acc: 0.5778
val Loss: 0.8843 Acc: 0.6667

Epoch 4/24
----------
train Loss: 0.9532 Acc: 0.6000
val Loss: 0.8284 Acc: 0.7333

Epoch 5/24
----------
train Loss: 0.9873 Acc: 0.5333
val Loss: 0.7648 Acc: 0.8000

Epoch 6/24
----------
train Loss: 0.8808 Acc: 0.7111
val Loss: 0.7248 Acc: 0.8667

Epoch 7/24
----------
train Loss: 0.9307 Acc: 0.5111
val Loss: 0.7141 Acc: 0.8667

Epoch 8/24
----------
train Loss: 0.8576 Acc: 0.6889
val Loss: 0.7262 Acc: 0.8667

Epoch 9/24
----------
train Loss: 0.8129 Acc: 0.7333
val Loss: 0.7144 Acc: 0.8667

Epoch 10/24
----------
train Loss: 0.8911 Acc: 0.6000
val Loss: 0.7038 Acc: 0.9333

Epoch 11/24
----------
train Loss: 0.8821 Acc: 0.6889
val Loss: 0.6936 Acc: 0.9333

Ep